In [1]:
# Build Graph
# graphframe
sc.addPyFile('/usr/local/Cellar/apache-spark/2.4.1/libexec/jars/graphframes-0.7.0-spark2.4-s_2.11.jar')

# modules
import sys
from graphframes import *
import pyspark.sql.functions as f
from pyspark.sql.functions import lit

# library 
sys.path.append("/Users/jeanxu/Documents/UniLU/0_MasterThesis/5_Notes/QA_Notes/1_Test/python/project/library")
import get_answer_types_from_questions as gtQType
import get_all_subjects_predicates_from_questions as gtQSub
from stanfordcorenlp import StanfordCoreNLP

In [46]:
import hearstPatterns as hP

In [2]:
# config
verbose = 0
nlp = StanfordCoreNLP(r'/Users/jeanxu/PycharmProjects/QUEST_TerminalTest/Code/stanford-corenlp-full-2018-10-05')
prune = 5
question = 'when is national day of England'

In [3]:
# Extracting potential answer type from question
f1 = 'question_type.txt'
gtQType.get_answer_type_main(question,f1,nlp)


In [4]:
# Extracting query terms from question.
s1 = 'question_subject_predicate.txt'
gtQSub.get_sub_pred_ques_main(question,s1,nlp)
aux_list = set()
for l in open('auxiliary_verb_list.txt'):
    l = l.strip()
    aux_list.add(l.lower())

In [5]:
# Read entities of question
s11 = open(s1, 'r')
q_ent = set()
type_qent = {}
for line in s11:
    line = line.strip()
    line = line.split()
    s = line[0]
    for i in range(1, len(line) - 1):
        s += ' ' + line[i]
    q_ent.add(s.lower())
    type_qent[s.lower()] = line[len(line) - 1]
if verbose:
    print "Query terms ->", len(q_ent), q_ent
if len(q_ent) > prune:
    if verbose:
        print "Pruning.."
    p = frozenset(q_ent)
    for s in p:
        if type_qent[s] == 'P':
            q_ent.remove(s)

In [6]:
q_ent

{'england', 'is', 'national day'}

In [7]:
type_qent

{'england': 'NE', 'is': 'P', 'national day': 'NE'}

In [8]:
p = frozenset(q_ent)
for s in p:
    if s in aux_list and type_qent[s] == 'P':
        q_ent.remove(s)

In [9]:
p

frozenset({'england', 'is', 'national day'})

In [10]:
# file path
File_verticesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Vertices/part-*"
File_edgesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Edges/part-*"


In [11]:
######### load vertices
verticesText0 = spark.read.csv(File_verticesTextRDD, header='false', inferSchema='false', sep='\t')

verticesText1 = verticesText0.select("_c1","_c0","_c2","_c3","_c4")\
.withColumnRenamed("_c0", "nodeType").withColumnRenamed("_c1", "id")\
.withColumnRenamed("_c2", "attr1").withColumnRenamed("_c3", "attr2")\
.withColumnRenamed("_c4", "attr3")

In [12]:
verticesText1.show()

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
| 10|Sentence|   11|Buyoya agreed in ...|Politics of Burundi|
| 11|Sentence|   12|Nonetheless, figh...|Politics of Burundi|
| 12|Sentence|   13|In June 1998, Buy...|Politics of Burundi|
| 13|Sen

In [13]:
######### load edges
edgesText0 = spark.read.csv(File_edgesTextRDD)
edgesText0 = edgesText0.select(f.translate(f.col("_c0"), "Edge(", "").alias("src"), "_c1", f.translate(f.col("_c2"), ")", "").alias("label"))
edgesText1 = edgesText0.select("*").withColumnRenamed("_c1", "dst")
verticesText1J = verticesText1

In [14]:
verticesText1J.show()

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
| 10|Sentence|   11|Buyoya agreed in ...|Politics of Burundi|
| 11|Sentence|   12|Nonetheless, figh...|Politics of Burundi|
| 12|Sentence|   13|In June 1998, Buy...|Politics of Burundi|
| 13|Sen

In [15]:
edgesText2 = edgesText1.join(verticesText1.select("id","nodeType"), edgesText1.src == verticesText1.select("id","nodeType").id, "inner")
edgesText2 = edgesText2.withColumnRenamed("id", "src_id").withColumnRenamed("nodeType", "src_nodeType")
edgesText3 = edgesText2.join(verticesText1.select("id","nodeType"), edgesText2.dst == verticesText1.select("id","nodeType").id, "inner")
edgesText3 = edgesText3.withColumnRenamed("id", "dst_id").withColumnRenamed("nodeType", "dst_nodeType")

In [16]:
edgesText3.show()

+-------+-------+-------------------+-------+------------+-------+------------+
|    src|    dst|              label| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+-------+-------------------+-------+------------+-------+------------+
|1234113| 100010|contains the clause|1234113|    Sentence| 100010|      Clause|
|3875588|1000240|contains the clause|3875588|    Sentence|1000240|      Clause|
|3650624|1000280|contains the clause|3650624|    Sentence|1000280|      Clause|
|2068926|1000665|contains the clause|2068926|    Sentence|1000665|      Clause|
| 467648|1000795|contains the clause| 467648|    Sentence|1000795|      Clause|
| 233944|1000839|contains the clause| 233944|    Sentence|1000839|      Clause|
|4022932|1000888|contains the clause|4022932|    Sentence|1000888|      Clause|
|1212133| 100140|contains the clause|1212133|    Sentence| 100140|      Clause|
| 452611|1001866|contains the clause| 452611|    Sentence|1001866|      Clause|
|1487892|1002011|contains the clause|148

In [ ]:
######### documnet vertice & edges

In [17]:
### documnets---attributes
doc_verticesText = verticesText1.filter("nodeType = 'Document'")
doc_verticesText = doc_verticesText.withColumn("did", doc_verticesText.id).withColumn("dtitle", doc_verticesText.attr1).drop("attr1").drop("attr2").drop("attr3")

In [18]:
doc_verticesText.show()

+----+--------+----+--------------------+
|  id|nodeType| did|              dtitle|
+----+--------+----+--------------------+
|4526|Document|4526|     Africa Alphabet|
|4527|Document|4527|               Brain|
|4528|Document|4528|     Andreas Aagesen|
|4529|Document|4529|Australian Broadc...|
|4530|Document|4530|The Elephant 6 Re...|
|4531|Document|4531|Albert of Branden...|
|4532|Document|4532|Telecommunication...|
|4533|Document|4533|                 DMA|
|4534|Document|4534|         Alford plea|
|4535|Document|4535|        Claude Piron|
|4536|Document|4536|Full disclosure (...|
|4537|Document|4537|    Anthem of Europe|
|4538|Document|4538|             Cytosol|
|4539|Document|4539|            Barbagia|
|4540|Document|4540|             Calcium|
|4541|Document|4541|   Chomsky hierarchy|
|4542|Document|4542| Critical philosophy|
|4543|Document|4543|   Cultural movement|
|4544|Document|4544|          Braveheart|
|4545|Document|4545|                Bill|
+----+--------+----+--------------

In [19]:
### sentence---attributes
# sen_verticesText = verticesText1.filter("nodeType = 'Sentence'")
sen_edges = edgesText3.filter("label = 'contains the sentence'")
sen_verticesTextJ_test = verticesText1.filter("nodeType = 'Sentence'")
sen_edgesJ = sen_edges.select("src_id","src_nodeType","dst_id","dst_nodeType")

In [20]:
sen_edgesJ.show()

+------+------------+-------+------------+
|src_id|src_nodeType| dst_id|dst_nodeType|
+------+------------+-------+------------+
|626404|    Document|1014942|    Sentence|
|626407|    Document|1015263|    Sentence|
|626411|    Document|1015551|    Sentence|
|626411|    Document|1015552|    Sentence|
|626411|    Document|1015610|    Sentence|
|626413|    Document|1015827|    Sentence|
|626414|    Document|1015968|    Sentence|
|626417|    Document|1016299|    Sentence|
|626417|    Document|1016518|    Sentence|
|626417|    Document|1016816|    Sentence|
|626418|    Document|1017045|    Sentence|
|626418|    Document|1017074|    Sentence|
|626419|    Document|1017462|    Sentence|
|626421|    Document|1017551|    Sentence|
|626422|    Document|1017716|    Sentence|
|626422|    Document|1017805|    Sentence|
|626422|    Document|1017849|    Sentence|
|626422|    Document|1018090|    Sentence|
|626422|    Document|1018100|    Sentence|
|626392|    Document|1018247|    Sentence|
+------+---

In [21]:
# conbine src_id
sen_edgesJ_test1 = sen_edgesJ.join(doc_verticesText, sen_edgesJ.src_id==doc_verticesText.id,"inner")
sen_edgesJ_test1 = sen_edgesJ_test1.drop("id").drop("nodeType")


In [22]:
sen_edgesJ_test1.show()

+-------+------------+-------+------------+-------+--------------+
| src_id|src_nodeType| dst_id|dst_nodeType|    did|        dtitle|
+-------+------------+-------+------------+-------+--------------+
|1019881|    Document|3245237|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245224|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245236|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245232|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245222|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245228|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245238|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245239|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245223|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245229|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245231|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245225|    Sentence|1019881|Emperor Nin

In [23]:
# conbine dst_id
sen_verticesTextJ_test1 = sen_edgesJ_test1.join(sen_verticesTextJ_test,sen_edgesJ_test1.dst_id==sen_verticesTextJ_test.id,"inner")
sen_verticesTextJ_test2 = sen_verticesTextJ_test1.drop("attr2").drop("attr3")
sen_verticesText = sen_verticesTextJ_test2.drop("src_id").drop("src_nodeType").drop("dst_id").drop("dst_nodeType")
sen_verticesText = sen_verticesText.withColumnRenamed("attr1","sid")
sen_verticesText = sen_verticesText.select("id","nodeType","did","dtitle","sid")

In [24]:
sen_verticesText.show()

+-------+--------+------+--------------------+---+
|     id|nodeType|   did|              dtitle|sid|
+-------+--------+------+--------------------+---+
|1014942|Sentence|626404|              Genome|102|
|1015263|Sentence|626407|             Alcohol| 99|
|1015551|Sentence|626411|             Cartoon|  5|
|1015552|Sentence|626411|             Cartoon|  6|
|1015610|Sentence|626411|             Cartoon| 64|
|1015827|Sentence|626413|Charlton Athletic...|190|
|1015968|Sentence|626414|          Battleship|135|
|1016299|Sentence|626417|          Bangladesh| 93|
|1016518|Sentence|626417|          Bangladesh|312|
|1016816|Sentence|626417|          Bangladesh|610|
|1017045|Sentence|626418|               Ghost| 40|
|1017074|Sentence|626418|               Ghost| 69|
|1017462|Sentence|626419|United States For...|133|
|1017551|Sentence|626421|         Amoxicillin| 38|
|1017716|Sentence|626422|         El Salvador|112|
|1017805|Sentence|626422|         El Salvador|201|
|1017849|Sentence|626422|      

In [27]:
### clause---attributes
clause_verticesText_test = verticesText1.filter("nodeType = 'Clause'")
clause_edges = edgesText3.filter("label = 'contains the clause'")


In [28]:
# combine dst_id
clause_verticesTextJ_test = clause_verticesText_test.join(clause_edges.select("src_id","src_nodeType","dst_id","dst_nodeType"), clause_verticesText_test.id==clause_edges.dst_id, "inner")
clause_verticesTextJ_test = clause_verticesTextJ_test.drop("src_nodeType").drop("dst_id").drop("dst_nodeType")


In [30]:
clause_verticesTextJ_test.show()

+-------+--------+--------------------+-----+--------------------+-------+
|     id|nodeType|               attr1|attr2|               attr3| src_id|
+-------+--------+--------------------+-----+--------------------+-------+
| 100010|  Clause|("Kerensky", "ord...|  SVO|            Kerensky|1234113|
|1000240|  Clause|("to the paternal...|  SVC|to the paternal h...|3875588|
|1000280|  Clause|("roughly speakin...|  SVO|roughly speaking ...|3650624|
|1000665|  Clause|("sympatric speci...|  SVC|sympatric speciation|2068926|
|1000795|  Clause|("The growth of t...|  SVO|The growth of the...| 467648|
|1000839|  Clause|("the priests", "...|  SVA|         the priests| 233944|
|1000888|  Clause|("the absolute va...|  SVC|  the absolute value|4022932|
| 100140|  Clause|("A memorial plaq...|  SVA|   A memorial plaque|1212133|
|1001866|  Clause|("1QDan", "is", "...|  SVC|               1QDan| 452611|
|1002011|  Clause|("the use of Afri...|  SVC|the use of Africa...|1487892|
|1002185|  Clause|("he", 

In [33]:
sen_verticesTextJ=sen_verticesText.withColumnRenamed("id","senid")##important, need to keep different column name with others when "JOIN"

In [35]:
# combine src_id
clause_verticesText_test1 = clause_verticesTextJ_test.join(sen_verticesTextJ.select("senid","did","dtitle","sid"), clause_verticesTextJ_test.src_id==sen_verticesTextJ.senid, "inner")
clause_verticesText_test2 = clause_verticesText_test1.drop("attr2").drop("attr3").drop("src_id").drop("sen_id")
clause_verticesText = clause_verticesText_test2

In [36]:
clause_verticesText.show()

+-------+--------+--------------------+-------+------+--------------------+---+
|     id|nodeType|               attr1|  senid|   did|              dtitle|sid|
+-------+--------+--------------------+-------+------+--------------------+---+
|2911087|  Clause|("Both the proces...|1014942|626404|              Genome|102|
|1732278|  Clause|("alcohols", "are...|1015263|626407|             Alcohol| 99|
|3236586|  Clause|("cartoon", "came...|1015551|626411|             Cartoon|  5|
|3493546|  Clause|("to animated fil...|1015552|626411|             Cartoon|  6|
|3493545|  Clause|("it", "began", "...|1015552|626411|             Cartoon|  6|
|3327830|  Clause|("Thomson", "capi...|1015610|626411|             Cartoon| 64|
|1230382|  Clause|("The ground-shar...|1015827|626413|Charlton Athletic...|190|
|2658479|  Clause|("the Austro-Hung...|1015968|626414|          Battleship|135|
|2658478|  Clause|("The Adriatic", ...|1015968|626414|          Battleship|135|
| 276441|  Clause|("British rule", ...|1

In [37]:
## change clause to predicate
clause_verticesText1 = clause_verticesText.withColumn("attr1", f.split(clause_verticesText['attr1'], ","))
predicate_verticesText = clause_verticesText.withColumn("predicate", f.split("attr1", ",")[1])
predicate_verticesText1 = predicate_verticesText.select("*", f.translate(f.col("predicate"), "[\"()]", "").alias("predicate1"))
predicate_verticesText2 = predicate_verticesText1.withColumn("nodeType1",f.lit("Predicate"))
predicate_verticesText = predicate_verticesText2.select("id","nodeType1","did","dtitle","sid","predicate1")

In [38]:
predicate_verticesText.show()

+-------+---------+------+--------------------+---+-----------------+
|     id|nodeType1|   did|              dtitle|sid|       predicate1|
+-------+---------+------+--------------------+---+-----------------+
|2911087|Predicate|626404|              Genome|102|       can result|
|1732278|Predicate|626407|             Alcohol| 99|              are|
|3236586|Predicate|626411|             Cartoon|  5|             came|
|3493546|Predicate|626411|             Cartoon|  6|        resembled|
|3493545|Predicate|626411|             Cartoon|  6|            began|
|3327830|Predicate|626411|             Cartoon| 64|      capitalized|
|1230382|Predicate|626413|Charlton Athletic...|190|              was|
|2658479|Predicate|626414|          Battleship|135|         remained|
|2658478|Predicate|626414|          Battleship|135|              was|
| 276441|Predicate|626417|          Bangladesh| 93|        displaced|
| 276440|Predicate|626417|          Bangladesh| 93|        broke out|
| 464234|Predicate|6

In [39]:
### mention---mention edges
entity_edges = edgesText3.filter("label = 'is disambiguated as'")
NewMentionEdges = entity_edges.groupBy("dst_id").agg(f.collect_list('src_id').alias('NewMentionEdges'))
NewMentionEdges = NewMentionEdges.withColumnRenamed("NewMentionEdges","Mention")
NewMentionEdges = NewMentionEdges.select("Mention")
NewMentionEdges = NewMentionEdges.withColumn("Mention1",NewMentionEdges.Mention)
NewMentionEdgesJ = NewMentionEdges.withColumn("Mention1", f.explode("Mention1"))

NewMentionEdgesJ1 = NewMentionEdgesJ.withColumn("Mention", f.explode("Mention"))
NewMentionEdgesJ2 = NewMentionEdgesJ1.filter(NewMentionEdgesJ1.Mention!=NewMentionEdgesJ1.Mention1)


In [40]:
NewMentionEdgesJ2.show()

+-------+--------+
|Mention|Mention1|
+-------+--------+
|4417364| 4417352|
|4417356| 4417352|
|7915519| 4417352|
|7171576| 4417352|
|7171574| 4417352|
|4417352| 4417364|
|4417356| 4417364|
|7915519| 4417364|
|7171576| 4417364|
|7171574| 4417364|
|4417352| 4417356|
|4417364| 4417356|
|7915519| 4417356|
|7171576| 4417356|
|7171574| 4417356|
|4417352| 7915519|
|4417364| 7915519|
|4417356| 7915519|
|7171576| 7915519|
|7171574| 7915519|
+-------+--------+
only showing top 20 rows



In [ ]:
##add type node

In [42]:
sen_verticesText_test=verticesText1.filter("nodeType = 'Sentence'")

In [43]:
sen_Type_verticesText_test=sen_verticesText_test.select("attr2")

In [44]:
sen_Type_verticesText_test.show()

+--------------------+
|               attr2|
+--------------------+
|Politics of Burun...|
|Executive power i...|
|Legislative power...|
|The political lan...|
|The current Presi...|
|Nkurunziza was th...|
|In November 1995,...|
|In July 1996, for...|
|He declared himse...|
|Widespread condem...|
|Buyoya agreed in ...|
|Nonetheless, figh...|
|In June 1998, Buy...|
|After facilitator...|
|Under Mandela the...|
|In April 2015 the...|
|Protests in the c...|
|The president is ...|
|He nominates two ...|
|The National Asse...|
+--------------------+
only showing top 20 rows



In [47]:
h = hP.HearstPatterns(extended=True)

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
replaceS_udf=udf(gtQSub.replace_symbols,StringType())

In [48]:
sen_Type_verticesText_test1 =sen_Type_verticesText_test.select(gtQSub.replace_symbols("attr2"))

In [49]:
sen_Type_verticesText_test1.show()

+--------------------+
|               attr2|
+--------------------+
|Politics of Burun...|
|Executive power i...|
|Legislative power...|
|The political lan...|
|The current Presi...|
|Nkurunziza was th...|
|In November 1995,...|
|In July 1996, for...|
|He declared himse...|
|Widespread condem...|
|Buyoya agreed in ...|
|Nonetheless, figh...|
|In June 1998, Buy...|
|After facilitator...|
|Under Mandela the...|
|In April 2015 the...|
|Protests in the c...|
|The president is ...|
|He nominates two ...|
|The National Asse...|
+--------------------+
only showing top 20 rows



In [55]:
s="this is an apple"

In [ ]:
s1=replace_symbols

In [56]:
h.find_hyponyms(s.decode('utf-8'))

[]

In [50]:
sen_Type_verticesText_test2 = sen_Type_verticesText_test1.select(h.find_hyponyms("apple"))

In [51]:
sen_Type_verticesText_test2.show()

++
||
++
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
||
++
only showing top 20 rows

